<a href="https://colab.research.google.com/github/wmjx691/3D-Urban-H-R-Diagram/blob/main/%E5%9F%8E%E5%B8%82%E8%B5%AB%E7%BE%85%E5%9C%96%E4%B8%89%E7%B6%AD%E8%A6%96%E8%A6%BA%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

這是一個非常棒的思維躍進！將這張圖表升維到 3D（XYZ 空間）確實更能完整描述「城市演化」的全貌。

在二維赫羅圖中，我們常常忽略了「半徑（人口）」這個變因，但在城市經濟學中，人口規模（Z軸）是極為關鍵的物理量。

我為您寫了一段 Python 程式碼，使用 **Plotly** 套件。這非常適合在 Google Colab 運行，因為它會生成一個**可互動的 3D 模型**，您可以自由旋轉、縮放，去觀察那些「偏離主序帶」的城市是如何分佈的。

### 城市赫羅圖 3D 視覺化程式碼 (Urban H-R Diagram 3D)

請將以下程式碼複製到您的 Google Colab 中執行：

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

# --- 1. 建立城市天體物理數據庫 ---
# 為了模擬真實情況，我們建立不同類型的城市數據
# 這些數值是基於現實的數量級估算 (Order of Magnitude)，經過對數平滑處理以利視覺化

data = [
    # --- 藍超巨星 (Blue Supergiants) ---
    # 高溫 (高人均), 高光度 (高總量), 大半徑 (大人口)
    {"City": "紐約 (NYC)", "Region": "Global", "Type": "Blue Supergiant", "GDP_Capita": 110000, "Total_GDP": 2000, "Population": 19},
    {"City": "東京 (Tokyo)", "Region": "Global", "Type": "Blue Supergiant", "GDP_Capita": 50000, "Total_GDP": 1800, "Population": 37},
    {"City": "倫敦 (London)", "Region": "Global", "Type": "Blue Supergiant", "GDP_Capita": 65000, "Total_GDP": 1000, "Population": 14},

    # --- 紅巨星 (Red Giants) ---
    # 低溫 (低人均), 高光度 (高總量), 超大半徑 (人口紅利支撐)
    {"City": "孟買 (Mumbai)", "Region": "Global", "Type": "Red Giant", "GDP_Capita": 6000, "Total_GDP": 300, "Population": 25},
    {"City": "雅加達 (Jakarta)", "Region": "Global", "Type": "Red Giant", "GDP_Capita": 10000, "Total_GDP": 400, "Population": 30},
    {"City": "新北市 (New Taipei)", "Region": "Taiwan", "Type": "Red Giant Branch", "GDP_Capita": 28000, "Total_GDP": 120, "Population": 4.0}, # 台灣內部的紅巨星傾向

    # --- 主序星 (Main Sequence) ---
    # 溫度與光度成正比，穩定的青壯年恆星
    {"City": "首爾 (Seoul)", "Region": "Global", "Type": "Main Sequence", "GDP_Capita": 40000, "Total_GDP": 900, "Population": 25},
    {"City": "台北市 (Taipei)", "Region": "Taiwan", "Type": "Main Sequence", "GDP_Capita": 55000, "Total_GDP": 160, "Population": 2.5},
    {"City": "墨爾本 (Melbourne)", "Region": "Global", "Type": "Main Sequence", "GDP_Capita": 50000, "Total_GDP": 250, "Population": 5.0},
    {"City": "台中 (Taichung)", "Region": "Taiwan", "Type": "Main Sequence", "GDP_Capita": 30000, "Total_GDP": 85, "Population": 2.8},
    {"City": "桃園 (Taoyuan)", "Region": "Taiwan", "Type": "Main Sequence", "GDP_Capita": 32000, "Total_GDP": 75, "Population": 2.3},
    {"City": "高雄 (Kaohsiung)", "Region": "Taiwan", "Type": "Main Sequence", "GDP_Capita": 28000, "Total_GDP": 70, "Population": 2.7},

    # --- 沃爾夫-拉葉星 / 藍色緻密矮星 (Wolf-Rayet / Blue Compact Dwarfs) ---
    # 極高溫 (極富), 光度中等 (受限於規模), 半徑小 (人口少)
    {"City": "竹北/新竹 (Hsinchu)", "Region": "Taiwan", "Type": "Blue Compact Dwarf", "GDP_Capita": 80000, "Total_GDP": 50, "Population": 0.5},
    {"City": "蘇黎世 (Zurich)", "Region": "Global", "Type": "Blue Compact Dwarf", "GDP_Capita": 100000, "Total_GDP": 150, "Population": 1.4},
    {"City": "聖荷西 (San Jose/Silicon Valley)", "Region": "Global", "Type": "Blue Compact Dwarf", "GDP_Capita": 130000, "Total_GDP": 300, "Population": 1.0},

    # --- 原恆星 / 金牛T星 (Protostars / T Tauri) ---
    # 溫度低 (還沒賺錢), 光度低, 正在吸積氣體
    {"City": "淡海新市鎮 (Danhai)", "Region": "Taiwan", "Type": "Protostar", "GDP_Capita": 20000, "Total_GDP": 10, "Population": 0.2},
    {"City": "青埔 (Qingpu)", "Region": "Taiwan", "Type": "Protostar", "GDP_Capita": 25000, "Total_GDP": 15, "Population": 0.15},

    # --- 白矮星 (White Dwarfs) ---
    # 溫度中高 (老本), 光度低 (規模萎縮), 半徑小 (人口流失)
    {"City": "基隆 (Keelung)", "Region": "Taiwan", "Type": "White Dwarf", "GDP_Capita": 26000, "Total_GDP": 10, "Population": 0.36},
    {"City": "底特律 (Detroit - Old)", "Region": "Global", "Type": "White Dwarf", "GDP_Capita": 28000, "Total_GDP": 80, "Population": 0.6}
]

df = pd.DataFrame(data)

# --- 2. 物理量轉換 (Log Scale for Visualization) ---
# 天文尺度通常跨度極大，使用對數座標才看得出結構
df['Log_GDP_Capita'] = np.log10(df['GDP_Capita'])
df['Log_Total_GDP'] = np.log10(df['Total_GDP'])
df['Log_Population'] = np.log10(df['Population'])

# --- 3. 繪製 3D 赫羅圖 ---
fig = px.scatter_3d(
    df,
    x='GDP_Capita',       # X軸: 表面溫度 (人均產值)
    y='Total_GDP',        # Y軸: 光度 (城市總產值)
    z='Population',       # Z軸: 半徑 (人口規模)
    color='Type',         # 顏色區分恆星光譜類型
    text='City',          # 顯示城市名稱
    size='Population',    # 點的大小也反映人口 (視覺輔助)
    symbol='Region',      # 符號區分台灣 vs 全球
    log_x=True,           # 全部採用對數座標
    log_y=True,
    log_z=True,
    title='3D Urban Hertzsprung-Russell Diagram (城市赫羅圖)',
    labels={
        'GDP_Capita': '溫度 T: 人均 GDP (USD, Log)',
        'Total_GDP': '光度 L: 城市總 GDP (Billion USD, Log)',
        'Population': '半徑 R: 人口 (Million, Log)'
    },
    height=800
)

# --- 4. 調整視角與軸向 (關鍵步驟) ---
# 赫羅圖的 X 軸 (溫度) 傳統上是「左高右低」 (Reversed)
fig.update_layout(
    scene=dict(
        xaxis=dict(autorange="reversed"), # 讓高溫 (富有) 在左邊
        yaxis=dict(title_font=dict(size=14)),
        zaxis=dict(title_font=dict(size=14)),
    ),
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.1)
)

# 顯示圖表
fig.show()

### 如何解讀這個三維模型？

當您在 Colab 運行出這個圖後，請嘗試旋轉它，您會看到我們剛剛討論的物理結構：

1. **主序帶 (The Main Sequence Plane)：**
* 您會發現「台北、首爾、墨爾本、台中」這幾個點，在三維空間中大致落在一個**斜切面**上。
* 這代表  的物理鐵律。大部分健康的城市都在這個平面上滑動。


2. **紅巨星分支 (Red Giant Branch)：**
* 請找「孟買」或「新北市」。
* 它們會**「浮」在 Z 軸的高處**（人口很多），Y 軸也不低（總量大），但 X 軸會偏向右側（溫度/人均較低）。
* **物理意義：** 它們是用「體積（人口）」硬撐出來的亮度，而不是靠核融合效率（高科技產業）。


3. **藍色緻密矮星 (Blue Compact Dwarfs)：**
* 請找「竹北/新竹」或「矽谷」。
* 它們會位於 X 軸的**極左側**（極度高溫），但 Z 軸很低（人口少）。
* **投資意義：** 這是能量密度最高的區域。如果它能吸積更多人口（Z軸上升），它就會變成藍巨星（像舊金山或台北市）。


4. **原恆星 (Protostars)：**
* 請找「淡海」或「青埔」。
* 它們躲在角落：低溫（右側）、低亮度（下方）、低半徑（下方）。
* **演化方向：** 它們的目標是往「左上角」移動。如果只往上（人口增加）但不往左（薪資不漲），就會變成失敗的褐矮星。



這張圖是您在評估房地產時的**「星圖導航儀」**。下次看重劃區，問問自己：它的軌跡是正朝向主序帶移動，還是正準備膨脹成虛胖的紅巨星？